In [1]:
import numpy as np
import os
import sys
import copy

import torch
torch.set_num_threads(os.cpu_count())
from torch import nn

import importlib
import json

import matplotlib as mpl
import matplotlib.pyplot as plt

from pipnet import data
from pipnet import model
from pipnet import utils

#import nmrglue as ng
import scipy
import scipy.io

device = "cuda" if torch.cuda.is_available() else "cpu"

np.random.seed(123)

In [2]:
model_name = "PIPNet_2022_12_09_2_layers"
model_dir = f"../../data/2D/{model_name}/"
fig_dir = f"../../figures/2D/{model_name}/"

in_dir = "../../data/2D/sanity_check/"
n_samples = 64

epoch = 250

In [3]:
if not os.path.exists(model_dir):
    raise ValueError(f"Unknown model: {model_name}")
    
if not os.path.exists(fig_dir):
    os.mkdir(fig_dir)
    
fdir = fig_dir + "eval_generated/"
if not os.path.exists(fdir):
    os.mkdir(fdir)

In [4]:
with open(model_dir + "model_pars.json", "r") as F:
    model_pars = json.load(F)
model_pars["noise"] = 0.

with open(model_dir + "data_pars.json", "r") as F:
    data_pars = json.load(F)

net = model.ConvLSTMEnsemble(**model_pars)
net.load_state_dict(torch.load(model_dir + f"epoch_{epoch}_network", map_location=device))
net = net.eval()

In [5]:
%%time

for isample in range(n_samples):
    
    print(f"Prediction {isample+1}/{n_samples}...")
    
    m = scipy.io.loadmat(f"{in_dir}generated_sample_{isample+1}.mat")
    ws = m["wr"][0]
    
    X = utils.prepare_2d_input(m["X"], ws, data_pars, xmax=1.)
    X[0, :, 0] = torch.Tensor(m["X"])
    
    with torch.no_grad():
        y_pred, _, _ = net(X)
    y_pred = y_pred.numpy()
    
    utils.plot_2d_iso_prediction(
        X[0].numpy(),
        y_pred[0],
        y_trg=m["iso"],
        wr_factor=data_pars["wr_norm_factor"],
        all_steps=False,
        show=False,
        level_neg=True,
        equal_axes=False,
        save=f"{fdir}generated_sample_{isample+1}.pdf"
    )
    
    utils.plot_2d_iso_prediction(
        X[0].numpy(),
        y_pred[0],
        y_trg=m["iso"],
        wr_factor=data_pars["wr_norm_factor"],
        all_steps=False,
        show=False,
        level_min=0.01,
        level_neg=True,
        equal_axes=False,
        save=f"{fdir}generated_sample_{isample+1}_low_contour.pdf"
    )
    
    output = {}
    output["X"] = X[0, :, 0].numpy()
    output["wr"] = ws
    output["pred"] = y_pred[0, -1]
    
    scipy.io.savemat(f"{fdir}generated_sample_{isample+1}_preds.mat", output)

print("All done!")

Prediction 1/64...
Prediction 2/64...
Prediction 3/64...
Prediction 4/64...
Prediction 5/64...
Prediction 6/64...
Prediction 7/64...
Prediction 8/64...
Prediction 9/64...
Prediction 10/64...
Prediction 11/64...
Prediction 12/64...
Prediction 13/64...
Prediction 14/64...
Prediction 15/64...
Prediction 16/64...
Prediction 17/64...
Prediction 18/64...
Prediction 19/64...
Prediction 20/64...
Prediction 21/64...
Prediction 22/64...
Prediction 23/64...
Prediction 24/64...
Prediction 25/64...
Prediction 26/64...
Prediction 27/64...
Prediction 28/64...
Prediction 29/64...
Prediction 30/64...
Prediction 31/64...
Prediction 32/64...
Prediction 33/64...
Prediction 34/64...
Prediction 35/64...
Prediction 36/64...
Prediction 37/64...
Prediction 38/64...
Prediction 39/64...
Prediction 40/64...
Prediction 41/64...
Prediction 42/64...
Prediction 43/64...
Prediction 44/64...
Prediction 45/64...
Prediction 46/64...
Prediction 47/64...
Prediction 48/64...
Prediction 49/64...
Prediction 50/64...
Predictio